# Scrapeo de Google

## Descripción de Tareas

Realizaremos scraping en el sitio web de Google Careers para obtener información sobre las vacantes de empleo.

1. **Scraping URL ("externalLink") vacantes**
   - Implementaremos una función para obtener las URLs de las vacantes.

2. **Scraping información de cada vacante**
   - Desarrollaremos una función para extraer información detallada de cada vacante.

3. **Transformación de datos (Spoiler deben quedar 41 COLUMNAS)**
   - Realizaremos la transformación de datos para asegurarnos de que haya 41 columnas en el resultado.

4. **Diccionario con información de la compañía**
   - Crearemos un diccionario con información específica de la compañía.

## Configuración Inicial

Realizaremos la configuración inicial del entorno y las importaciones necesarias.


In [ ]:
import os
import json
import requests
from bs4 import BeautifulSoup # Para analizar el HTML de las páginas web
from pydantic import BaseModel 
from typing import List, Optional
import pandas  as pd

URL de la Página de Google Careers


## Schema

In [ ]:
from __future__ import annotations
from typing import Any, List, Optional
from pydantic import BaseModel,Extra

class Skill(BaseModel):
    name: str
    level: str
    experience: int

class Aptitude(BaseModel):
    name: str

class Tool(BaseModel):
    name: str
    level: str
    experience: int

class Language(BaseModel):
    name: str
    level: str

class Benefit(BaseModel):
    name: str

class JobPost(BaseModel):
    # id: Optional[int] = None
    # companyId: Optional[str] = None
    name: Optional[str] = None
    description: Optional[str] = None
    createdAt: Optional[str] = None
    availableSlots: Optional[int] = None
    skills: Optional[List[Skill]] = None
    aptitudes: Optional[List[Aptitude]] = None
    tools: Optional[List[Tool]] = None
    languages: Optional[List[Language]] = None
    benefits: Optional[List[Benefit]] = None
    scholarity: Optional[str] = None
    workhours: Optional[str] = None
    locationConditions: Optional[str] = None
    nationalRemote: Optional[Any] = None
    minSalary: Optional[int] = None
    maxSalary: Optional[int] = None
    minAge: Optional[int] = None
    maxAge: Optional[Any] = None
    sex: Optional[str] = None
    yearsOfExperience: Optional[int] = None
    status: Optional[str] = None
    country: Optional[str] = None
    updatedAt: Optional[str] = None
    driversLicense: Optional[bool] = None
    degree: Optional[bool] = None
    validPassport: Optional[bool] = None
    validVisa: Optional[bool] = None
    nationalRelocation: Optional[bool] = None
    internationalRelocation: Optional[bool] = None
    availabilityToTravel: Optional[bool] = None
    seniority: Optional[str] = None
    showSalaryRange: Optional[bool] = None
    state: Optional[str] = None
    city: Optional[str] = None
    postalCode: Optional[str] = None
    slug: Optional[str] = None
    latitude: Optional[float] = None
    longitude: Optional[float] = None
    companyName: Optional[str] = None
    companyImg: Optional[str] = None
    class Config:
        extra = Extra.ignore


1. Scraping URL de Vacantes

In [ ]:
# URL de la página de Google Careers
base_url = "https://www.google.com/about/careers/applications/"
request_url = "https://www.google.com/about/careers/applications/jobs/results/?utm_campaign=profilepage&utm_medium=profilepage&utm_source=linkedin&src=Online%2FLinkedIn%2Flinkedin_page&location=Mexico"

def get_soup(url):
    response = requests.get(url)

    if response.status_code  == 200:
        return BeautifulSoup(response.content, 'html.parser')

In [ ]:
def scrape_vacancy_urls():
    soup = get_soup(request_url)
    urls = []
    
    for posteo in soup.find_all('div', class_='sMn82b'):
        name = posteo.text.strip()
        link_tag = posteo.find('a')  # Buscar el elemento <a> dentro de posteo
        if link_tag:
            link = base_url + link_tag.get('href')  # Obtener el valor del atributo 'href'
            urls.append({"name": name, "link": link})
    
    return urls

Función para guardar datos en formato JSON

In [ ]:
def save_to_json(data, filename):
    # Obtén la ruta completa del archivo en el directorio 'data'
    file_path = os.path.join('data', filename)
    
    # Crea el directorio 'data' si no existe
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=2)


In [ ]:
url_vacantes = scrape_vacancy_urls()
save_to_json(url_vacantes, 'url_vacantes.json')
print("Vacantes guardadas en url_vacantes.json")

2. **Scraping información de cada vacante**
   - Desarrollaremos una función para extraer información detallada de cada vacante.

In [ ]:
def scrape_vacancy_details(request_url):
    soup = get_soup(request_url)
    job_post = JobPost()

    # Extraer información detallada de cada vacante
    title_elem = soup.find("h2", class_="p1N2lc")
    location_elem = soup.find("span", class_="r0wTof").find("span", class_="pwO9Dc vo5qdf")
    requirements_elem = soup.find("div.KwJkGe > ul > li")   
    responsibilities_elem = soup.find("div", class_="BDNOWe").find("ul").find("li")

    # Asignar valores a las propiedades del objeto JobPost
    job_post.name = title_elem.text.strip() if title_elem else "No disponible"
    job_post.locationConditions = location_elem.text.strip() if location_elem else "No disponible"
    job_post.aptitudes = requirements_elem.text.strip() if requirements_elem else "No disponible"
    job_post.skills = responsibilities_elem.text.strip() if responsibilities_elem else "No disponible"

    # Convertir JobPost a json
    formato_json = {
        "name" : job_post.name,
        "location" : job_post.locationConditions,
        "requirements": job_post.aptitudes,
        "responsabilities": job_post.skills
    }

    return formato_json

Función para guardar datos en formato JSON

In [ ]:
def save_to_json(data, filename):    
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=2)

In [ ]:
detalles = []

with open('data/url_vacantes.json', 'r') as archivo:
    vacantes = json.load(archivo)

# Iterar sobre la lista de diccionarios en el archivo JSON
for vacante in vacantes:
    # Obtener la URL de cada trabajo
    url = vacante['link']
    informacion_vacantes= scrape_vacancy_details(url)
    
    detalles.append(informacion_vacantes)

save_to_json(detalles, 'data/informacion_vacantes.json')
print("Vacantes guardadas en informacion_vacantes.json")

4. **Diccionario con información de la compañía**
   - Crearemos un diccionario con información específica de la compañía.



In [ ]:

diccionario_google = {
    "userId": "google",
    "name": "Google",
    "description": """
                    Google es una empresa multinacional de tecnología estadounidense que se especializa en servicios y productos relacionados con Internet, incluyendo búsqueda en línea, publicidad, computación en la nube, software y hardware. 
                    Fundada en 1998, Google ha crecido hasta convertirse en una de las mayores empresas de tecnología del mundo. Su misión es organizar la información del mundo y hacerla universalmente accesible y útil.
                    Google se esfuerza por crear tecnologías innovadoras que mejoren la vida de las personas y fomenten la accesibilidad a la información. La empresa valora la diversidad, la inclusión y el impacto positivo en la sociedad.
                    """,
    "website": "https://www.google.com/",
    "sector": "Tecnología",
    "isExternal": True,
}

# DataFrame de Google
df_google = pd.DataFrame(diccionario_google, index=[0])
